# Building a single tree

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df_soybean = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/soybean/soybean-large.data',header=None)

307 cases, 35 variables (observation on plants with some climatic variables) and 19 classes (soybean diseases)

Attribute Information:

1. date: april,may,june,july,august,september,october,?.
2. plant-stand: normal,lt-normal,?.
3. precip: lt-norm,norm,gt-norm,?.
4. temp: lt-norm,norm,gt-norm,?.
5. hail: yes,no,?.
6. crop-hist: diff-lst-year,same-lst-yr,same-lst-two-yrs,
same-lst-sev-yrs,?.
7. area-damaged: scattered,low-areas,upper-areas,whole-field,?.
8. severity: minor,pot-severe,severe,?.
9. seed-tmt: none,fungicide,other,?.
10. germination: 90-100%,80-89%,lt-80%,?.
11. plant-growth: norm,abnorm,?.
12. leaves: norm,abnorm.
13. leafspots-halo: absent,yellow-halos,no-yellow-halos,?.
14. leafspots-marg: w-s-marg,no-w-s-marg,dna,?.
15. leafspot-size: lt-1/8,gt-1/8,dna,?.
16. leaf-shread: absent,present,?.
17. leaf-malf: absent,present,?.
18. leaf-mild: absent,upper-surf,lower-surf,?.
19. stem: norm,abnorm,?.
20. lodging: yes,no,?.
21. stem-cankers: absent,below-soil,above-soil,above-sec-nde,?.
22. canker-lesion: dna,brown,dk-brown-blk,tan,?.
23. fruiting-bodies: absent,present,?.
24. external decay: absent,firm-and-dry,watery,?.
25. mycelium: absent,present,?.
26. int-discolor: none,brown,black,?.
27. sclerotia: absent,present,?.
28. fruit-pods: norm,diseased,few-present,dna,?.
29. fruit spots: absent,colored,brown-w/blk-specks,distort,dna,?.
30. seed: norm,abnorm,?.
31. mold-growth: absent,present,?.
32. seed-discolor: absent,present,?.
33. seed-size: norm,lt-norm,?.
34. shriveling: absent,present,?.
35. roots: norm,rotted,galls-cysts,?.

[https://archive.ics.uci.edu/ml/datasets/soybean+(small)]

### Replace all "?" in table with NaN which will be replaced by the rounded column mean after each train-test split

In [ ]:
df_to_use= df_soybean
df_to_use = df_to_use.replace('?', np.NaN)
df_to_use

In [ ]:
### If replace_after_split is False, the error rates are closer the the paper's result
### The paper did not explicitly state when to replace missing values, 
### but it seems like best practice is to replace missing values after the train-test split.
replace_after_split = True

In [ ]:
if replace_after_split==False:
    for i in range(1,36):
         df_to_use[i] = df_to_use[i].fillna(round(df_to_use[i].dropna().astype(float).mean(),0))

In [ ]:
### Y is the category column and X is the matrix with all other columns
X = df_to_use.values[:, 1:36]
Y = df_to_use.values[:,0]#.astype('category')

In [ ]:
### Viewing the Classification Column mapped to values
df_to_use.iloc[0]

In [ ]:
### Number of unique classes (Y values)
#df_to_use.values[:,0].unique()
print(len(df_to_use[0].unique()))
df_to_use[0].unique()

In [ ]:
X

In [ ]:
i_max = 100 # length of matrix (number repeats)
j_max = 50 # width of matrix (bootstrap replicates)

In [ ]:
# err_0 is the base (without bagging)
cases = len(df_to_use)
err_0 = np.zeros((i_max,1)) #ToDo just confirm
for i in range(i_max):
    
    for j in range(1):
        print(i, j, end="\r")
        df_to_use_sample = df_to_use
        X = df_to_use_sample.values[:, 1:35]
        Y = df_to_use_sample.values[:, 0]#.astype('category')
        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3) # 70% training and 30% test
        X_train = X_train.astype(float)
        X_test = X_test.astype(float)
        # replace missing values with rounded mean (after train/test split)
        # https://stackoverflow.com/questions/18689235/numpy-array-replace-nan-values-with-average-of-columns
        col_means = np.nanmean(X_train, axis=0)
        inds = np.where(np.isnan(X_train))
        X_train[inds] = np.take(col_means, inds[1])
        ### Rounding, because all values are categorical, not numerical
        col_means = np.round(np.nanmean(X_test, axis=0))
        inds = np.where(np.isnan(X_test))
        X_test[inds] = np.take(col_means, inds[1])
        
        # Create Decision Tree classifer object
        clf = DecisionTreeClassifier()
        # Train Decision Tree Classifer
        clf = clf.fit(X_train,y_train)
        #Predict the response for test dataset
        y_pred = clf.predict(X_test)
        for k in range(0,len(y_test)-1):
            if y_test[k] != y_pred[k]:
                err_0[i,j] = err_0[i,j] + 1
        err_0[i,j]= err_0[i,j]/len(y_test)

In [ ]:
### Viewing the base
err_0

In [ ]:
# err is the method in the paper with bagging
cases = len(df_to_use)
err = np.zeros((i_max,j_max))
for i in range(i_max):
    
    for j in range(j_max):
        print(i, j, end="\r")
        df_to_use_sample = df_to_use.sample(n=cases, replace=True)
        X = df_to_use_sample.values[:, 1:35]
        Y = df_to_use_sample.values[:, 0]#.astype('category')
        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3) # 70% training and 30% test
        X_train = X_train.astype(float)
        X_test = X_test.astype(float)
        # replace missing values with mean (after train/test split)
        # https://stackoverflow.com/questions/18689235/numpy-array-replace-nan-values-with-average-of-columns
        col_means = np.nanmean(X_train, axis=0)
        inds = np.where(np.isnan(X_train))
        X_train[inds] = np.take(col_means, inds[1])
        ### Rounding, because all values are categorical, not numerical
        col_means = np.round(np.nanmean(X_test, axis=0))
        inds = np.where(np.isnan(X_test))
        X_test[inds] = np.take(col_means, inds[1])
        # Create Decision Tree classifer object
        clf = DecisionTreeClassifier()
        # Train Decision Tree Classifer
        clf = clf.fit(X_train,y_train)
        #Predict the response for test dataset
        y_pred = clf.predict(X_test)
        for k in range(0,len(y_test)-1):
            if y_test[k] != y_pred[k]:
                err[i,j] = err[i,j] + 1
        err[i,j]= err[i,j]/len(y_test)

In [ ]:
### Viewing err
err

## The paper found 14% for err0 and 10% for err, which is the improvement from bagging

In [ ]:
print(err_0.mean())
plt.hist(err_0, bins = 60)
plt.show()

In [ ]:
print(err.mean())
import matplotlib.pyplot as plt
plt.hist(err)
plt.show()

In [ ]:
plt.hist(np.concatenate(err_0), bins = 30)
plt.show()

In [ ]:
plt.hist(np.concatenate(err))
plt.show()

In [ ]:
fig, ax1 = plt.subplots()
ax1.hist(np.concatenate(err_0), color='tab:red', alpha=0.5, label='base - avg: {}'.format(round(err_0.mean(), 3)))
ax1.tick_params(axis='y', labelcolor='tab:red')

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax2.hist(np.concatenate(err),alpha=0.5, label='bagging - avg: {}'.format(round(err.mean(), 3)))
ax2.tick_params(axis='y', labelcolor='tab:blue')

fig.legend(loc='upper right')
fig.show()